# keras CNN

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [2]:
from keras.layers import Dense, Activation
from keras.layers import Convolution2D, MaxPooling2D, Flatten

import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
# import tensorflow as tf

# config = tf.ConfigProto(allow_soft_placement=True)
# config.gpu_options.allocator_type = 'BFC'
# config.gpu_options.per_process_gpu_memory_fraction = 0.40
# config.gpu_options.allow_growth=True

## Data Load

In [4]:
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# normalize 归一化
# 注意：这里如果不归一化，效果特别差
X_train = X_train.reshape((-1, 1, 28, 28))/255.
X_test = X_test.reshape((-1, 1, 28, 28))/255.

# one-hot 编码
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

print(X_train[1].shape)
print(y_train[0])

print()
print(X_train.shape)
print(y_train.shape)

(1, 28, 28)
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]

(60000, 1, 28, 28)
(60000, 10)


## build the CNN

In [5]:
model = Sequential()

# conv layer 1 output shape(32, 28, 28)
model.add(
    Convolution2D(
        filters=32,     # number of filter, 滤波器的个数（卷积核）
        kernel_size=5,  # number of row， 滤波器的大小
        strides=1,
        padding='same',  # padding method
        batch_input_shape=(None, 1, 28, 28),  # picture height & width
        data_format='channels_first',)
)

model.add(Activation('relu'))

# poolint layer 1(max pooling output shape(64, 7, 7))
model.add(
    MaxPooling2D(
        pool_size=2, 
        strides=2,
        padding='same',  # padding method
        data_format='channels_first',)
)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# conv layer 2 output shape(64, 14, 14)
model.add(
    Convolution2D(
        filters=64,  # number of filter, 滤波器的个数（卷积核）
        kernel_size=5,  # number of row， 滤波器的大小
        strides=1,
        padding='same',  # padding method
        data_format='channels_first',
))

model.add(Activation('relu'))
model.add(
    MaxPooling2D(
        pool_size=2, 
        strides=2, 
        padding='same', 
        data_format='channels_first',)
)

In [7]:
# fully conneted layer 1 input shape(64*7*&7)=(3136), output shape(1024)
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))

# fully conneted layer 2 to shape(10) for 10 classes
model.add(Dense(10))
model.add(Activation('softmax'))

## Another  to define your optimizer

In [18]:
adam = Adam(lr=1e-4)

model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

## Train

In [9]:
%%time
print('Training ----------------------')
model.fit(X_train, y_train, epochs=1, batch_size=32)

Training ----------------------
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
60000/60000 [==============================] - 332s 6ms/step - loss: 0.2048 - acc: 0.9419
Wall time: 5min 32s


## Test

In [67]:
%%time
print('Testing ----------------------')

# evaluate the model with the metrics we defined earlier[accuracy]
loss, accuracy = model.evaluate(X_test, y_test)

print('\n')
print('test loss:\t', loss)
print('test accuracy:\t', accuracy)

Testing ----------------------
10000/10000 [==============================] - 25s 3ms/step


test loss:	 0.20426653486080468
test accuracy:	 0.9374
Wall time: 25.2 s


## 实验结果说明
- 训练时候的batch_size设置为32比64要好

> batch_size=32: 0.9753

> batch_size=64: 0.9686

- 数据预处理时候，一定要归一化。对实验结果影响超级大

## 模型结构输出

In [9]:
from keras.utils import plot_model

plot_model(model, to_file='../pic/cnn_model.png', show_shapes=True, show_layer_names=True)

## 使用tensorboard
tensorboard --logdir=.\

In [9]:
import time
from tensorflow.keras.callbacks import TensorBoard

In [10]:
model_name = "andyni-test-cnn-%d" % int(time.time())
model_name

'andyni-test-cnn-1561017485'

In [44]:
tensorboard = TensorBoard(
    log_dir='../log/%s' % model_name, 
    
    
#     batch_size=32,
#     update_freq=20,
#     histogram_freq=1, 
#     write_grads=1,
#     write_images=1, 
#     write_graph=True, 

#     histogram_freq=1, 
    batch_size=32,
    update_freq=50,
    write_graph=True, 
    write_grads=True, 
    write_images=True,
    embeddings_freq=0, 
    embeddings_layer_names=None,
    embeddings_metadata=None,
    embeddings_data=None, 
)

tensorboard

In [45]:
print(X_train.shape, X_train[:5000, :].shape)

(60000, 1, 28, 28) (5000, 1, 28, 28)


In [79]:
# %%time
print('Training ----------------------')

history_data = model.fit(
    X_train[:100, :], y_train[:100, :], 
#     validation_data=(X_test[:500], y_test[:500]),
    epochs=5,
    batch_size=32, 
    validation_split=0.2, 
#     shuffle=True, 
#     callbacks=[tensorboard],
    verbose=1,
)

Training ----------------------
Train on 80 samples, validate on 20 samples
Epoch 1/5
80/80 [==============================] - 1s 10ms/step - loss: 0.0102 - acc: 1.0000 - val_loss: 0.0026 - val_acc: 1.0000
Epoch 2/5
80/80 [==============================] - 1s 9ms/step - loss: 0.0094 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 1.0000
Epoch 3/5
80/80 [==============================] - 1s 10ms/step - loss: 0.0082 - acc: 1.0000 - val_loss: 0.0020 - val_acc: 1.0000
Epoch 4/5
80/80 [==============================] - 1s 12ms/step - loss: 0.0073 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000
Epoch 5/5
80/80 [==============================] - 1s 9ms/step - loss: 0.0066 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000


In [63]:
history = model.fit(
    X_train[:1000, :], y_train[:1000, :], 
    validation_split=0.33, 
    epochs=1, 
    batch_size=10, 
    verbose=1,
) # list all data in history


Train on 669 samples, validate on 331 samples
Epoch 1/1
669/669 [==============================] - 18s 26ms/step - loss: 0.0212 - acc: 1.0000 - val_loss: 0.1052 - val_acc: 0.9698


## 训练过程数据 - keras.callbacks.History

In [82]:
type(history_data)

keras.callbacks.History

In [83]:
history_data.epoch

[0, 1, 2, 3, 4]

In [84]:
history_data.params

{'batch_size': 32,
 'epochs': 5,
 'steps': None,
 'samples': 80,
 'verbose': 1,
 'do_validation': True,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc']}

In [87]:
history_data.history

{'val_loss': [0.0026031925808638334,
  0.002168559469282627,
  0.0019702825229614973,
  0.0019085370004177094,
  0.0019146029371768236],
 'val_acc': [1.0, 1.0, 1.0, 1.0, 1.0],
 'loss': [0.01024192590266466,
  0.00938895419239998,
  0.00823008194565773,
  0.007289296155795455,
  0.0066370277665555475],
 'acc': [1.0, 1.0, 1.0, 1.0, 1.0]}